In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.special import zeta

In [ ]:
from astropy import units as u
from astropy.modeling import models

In [ ]:
nombre_alumno = "Francisco Montenegro"

## Constantes físicas importadas de Astropy (Sistema Internacional)
(ver https://docs.astropy.org/en/stable/constants/index.html)

In [ ]:
from astropy.constants import c,h,k_B

In [ ]:
print(c.si)

In [ ]:
print(h.si)

In [ ]:
print(k_B.si)

## Diccionarios con los intervalos de frecuencia/longitud de onda por banda del espectro

In [ ]:
vis_color_ranges = {'red':[625*u.nm,750*u.nm],
               'orange':[590*u.nm,625*u.nm],
               'yellow':[565*u.nm,590*u.nm],
               'green':[500*u.nm,565*u.nm],
               'cyan':[485*u.nm,500*u.nm],
               'blue':[450*u.nm,485*u.nm],
               'violet':[380*u.nm,450*u.nm]
               }

nir_color_ranges = {'J':[1170*u.nm,1330*u.nm],
                    'H':[1490*u.nm,1780*u.nm],
                    'K':[2030*u.nm,2370*u.nm]
                }

color_dict = {'red':'red',
               'orange':'orange',
               'yellow':'yellow',
               'green':'green',
               'cyan':'cyan',
               'blue':'blue',
               'violet':'violet',
              'J':'darkred',
              'H':'maroon',
              'K':'brown'
}

## Funciones auxiliares

In [ ]:
def lambda2nu(lam):
    return (c/lam).value 

def nu2lambda(nu):
    return (c/nu).value

def custom(name):
    return 10*ord(name[0])-ord(name[-1])

def relative_delta(a1,a2):
    """ Given an interval [a1,a2] it returns the interval width (in percentage) 
    referred to the mid point in the interval.
    """
    interval = a2-a1
    mid_point = 0.5*(a1+a2)
    return 100*interval/mid_point

def generate_lambda_vector():
    """ We generate a vector with wavelenghts that can be used over several decades in frequency (log scale)
    """
    return np.array([np.power(10,(1.5*i-1000)/78) for i in range(950)])*u.m

def generate_nu_vector():
    """ We generate a vector with frequencies that can be used over several decades (log scale)
    """
    return np.array([np.power(10,(i-9)/58.0) for i in range(1000)])*u.Hz

def integrand3(x):
    """ Special integral related to Riemmann's zeta function
    """
    return x**3/(np.exp(x)-1)

def integrand2(x):
    """ Special integral related to Riemmann's zeta function
    """
    return x**2/(np.exp(x)-1)

In [ ]:
r = custom(nombre_alumno) 
print(r)

## Ley de Planck para la emisión de un cuerpo negro

La radiancia espectral (energía emitida por unidad de tiempo, por unidad de ángulo sólido y por unidad de frecuencia) de un cuerpo negro a temperatura T viene dada por:

$$B_{\nu}(T) = \frac{2h\nu^3}{c^2} \frac{1}{e^{h\nu/kT}-1} \qquad  [W m^{-2} Hz^{-1} sr^{-1}]$$ 

Si queremos expresar la radiancia espectral en función de la longitud de onda, no podemos simplemente sustituir $\nu=c/\lambda$ en la formula anterior, ya que $B_{\nu}$ y $B_{\lambda}$ son magnitudes diferentes con unidades diferentes. Se relacionan teniendo en cuenta que dado un incremento espectral $d\lambda$, el incremento de energía radiada es el mismo tanto expresado en $\lambda$ como en $\nu$ pero con signo cambiado por ser ambas magnitudes recíprocas ($\lambda\,\nu$ = c):

$$B_{\lambda}\,d\lambda = -B_{\nu}\,d\nu $$

Por tanto en función de $\lambda$ tendremos:

$$B_{\lambda}(T) = \frac{2hc^2}{\lambda^5} \frac{1}{e^{hc/kT\lambda}-1} \qquad  [W m^{-2} m^{-1} sr^{-1}]$$ 

##### <font color='red'>Para hacer (a)</font>
Obtener B$_{\lambda}$ a partir de B$_{\nu}$ (o al revés), a partir de la igualdad anterior (entregar en papel).

In [ ]:
def planck_law(nu,T):
    """ Spectral radiance (intensity) by a Black body at temperature T, as a function of frequency
    """
    num = 2*h*np.power(nu,3)/np.power(c,2)
    denom = np.exp(h*nu/(k_B*T))-1
    return (num/denom)/u.sr

def planck_law_lam(lam,T):
    """ Spectral radiance (intensity) by a Black body at temperature T, as a function of wavelength
    """
    num = 2*h*np.power(c,2)/np.power(lam,5)
    denom = np.exp(h*c/(k_B*T*lam))-1
    return (num/denom)/u.sr 

#### Parte 1. 
Vamos a comprobar que la energía radiada en un incremento espectral pequeño $[\lambda_{1}, \lambda_{1}+\delta\lambda]$ es la misma utilizando las fórmulas anteriores de $B_{\lambda}(T)$ y de $B_{\nu}(T)$.

Calcular para qué valor *relativo* de $\delta\lambda$ conseguimos que la enegía radiada se asemeje con ambas expresiones hasta en un valor menor de $10^{-3}$:

In [ ]:
# Definimos la temperatura de una estrella (personalizada) que asumiremos emite como cuerpo negro
# r es un valor diferente para cada uno de vosotros que hace cada una de vuestras estrellas única

T_mystar = (5800+r)*u.K
print(T_mystar)

In [ ]:
# Elegimos una longitud de onda de referencia donde mediremos nuestro intervalo, corresponde al color amarillo
lambda1 = 580*u.nm  

##### <font color='red'>Para hacer (b)</font>
Disminuir el ancho de $\delta\lambda$ hasta que $B_{\lambda}\,\delta\lambda$ se aproxime a $B_{\nu}\,\delta\nu$ en menos de $10^{-3}$. 

In [ ]:
# Empezamos probando con un intervalo de 100 nm y vamos bajando hasta llegar lo más cerca posible debajo de 1e-9
delta_lambda = 100*u.nm

In [ ]:
lambda2 = lambda1 + delta_lambda
# Longitud de onda central del intervalo
mid_lambda = lambda1 + delta_lambda/2.0

# Calculamos el correspondiente intervalo escrito en frecuencias:
nu1 = c/lambda1
nu2 = c/lambda2
delta_nu = nu2-nu1  
mid_nu = (nu1 + nu2)/2.0

print(f"El intervalo en lambda es: [{lambda1:6.2f}, {lambda2:6.2f}]")
print(f"El intervalo en nu es: [{nu1.to('THz'):6.2f}, {nu2.to('THz'):6.2f}]")

In [ ]:
# Calculamos el ancho del intervalo espectral en relación con la longitud de onda central
relative_width_lambda = relative_delta(lambda1,lambda2)
print(f"El ancho del intervalo relativo a la longitud de onda central es de {relative_width_lambda:7.5f} %")
relative_width_nu = relative_delta(nu1,nu2)
print(f"El ancho del intervalo relativo a la frecuencia central es de {relative_width_nu:7.5f} %")

##### <font color='red'>Para hacer (b)</font>

Evaluar las funciones \*planck_law_lam\* y \*planck_law\* con los parámetros correspondientes. Sustituir las XXX

In [ ]:
# Evaluamos la función de B_lambda en la mitad del intervalo y multiplicamos por el intervalo.
e1=planck_law_lam(XXXX,XXXX)*delta_lambda 
e1 = e1.to('W / (m2 sr)')

# Evaluamos la función B_nu en la mitad del intervalo y multiplicamos por el intervalo
e2=planck_law(XXXX,XXXX)*delta_nu
e2 = e2.to('W / (m2 sr)')

print(f"B_lambda * delta_lambda = {e1}")
print(f"B_nu * delta_nu = {e2}")

In [ ]:
diff = abs(e1+e2)    
diff

In [ ]:
diff.value < 1e-3




---





#### Parte 2. Vamos a visualizar la distribución espectral correspondiente a nuestra estrella/cuerpo-negro

In [ ]:
# Generamos un vector de longitudes de onda que nos sirva para una escala logrítmica
lam = generate_lambda_vector()
lam

##### <font color='red'>Para hacer (c)</font>

Calcularemos la curva de Planck para nuestro vector de lambda. Luego calcularemos la longitud de onda del máximo de la curva, mediante la Ley de desplazamiento de Wien. Dicha ley nos dice la temperatura del cuerpo negro es inversamente proporcional a la longitud de onda del máximo. 

$$ \lambda_{max} T = b$$

Tendremos que importar de astropy la constante b (o como se llame) y usarla en la siguiente ecuación.

In [ ]:
# Calculamos la curva de emisividad según la ley de Planck

B_lam = planck_law_lam(lam,T_mystar)

# De acuerdo a la ley de Wien: https://es.wikipedia.org/wiki/Ley_de_desplazamiento_de_Wien

b = XXXXXX 
lam_peak = (b/T_mystar)

print(f"Lambda(peak) = {lam_peak.to('nm'):5.1f}")

In [ ]:
# Representamos la curva en función de la longitud de onda

fig, ax = plt.subplots(figsize=(12,8))

plt.plot(lam,B_lam,label=f'My Star ({T_mystar})',color='orange')

plt.title(f"Emission from {nombre_alumno}'s star")
ax.set_xlabel(r'$\lambda$ [m]',fontsize=14)
ax.set_ylabel(r'Radiance $\,$ B$_{\lambda}$ [W sr$^{-1}$ m$^{-1}$ m$^{-2}$]',fontsize=14)
plt.legend()

# Pintamos los colores del visible para ver donde cae el pico de emisión de nuestra estrella
for k, v in vis_color_ranges.items():
    lammin=(v[0].to('m')).value
    lammax=(v[1].to('m')).value
    plt.axvspan(xmin=lammin,xmax=lammax,color=color_dict[k],alpha=0.3)

# Pintamos aproximadamente las bandas del infrarrojo cercano (J, H, K)

for k, v in nir_color_ranges.items():
    lammin=(v[0].to('m')).value
    lammax=(v[1].to('m')).value
    plt.axvspan(xmin=lammin,xmax=lammax,color=color_dict[k],alpha=0.3)
    
    
# Marcamos la longitud de onda máxima de la curva de Planck
plt.axvline(lam_peak.to("m").value, ls='--',color='black')
    
# Restringimos el eje X al rango adecuado de longitud de onda que encuadra nuestra figura

lambda_min_plot = (10*u.nm).to('m').value
lambda_max_plot = (3*u.um).to('m').value

ax.set_xlim([lambda_min_plot,lambda_max_plot])

## Aproximación a baja energía (Rayleigh-Jeans)

Se cumplirá para los fotones menos energéticos emitidos por el cuerpo negro, para los cuales $h\nu << kT$. Si hacemos $x = \frac{h\nu}{kT}$ esto es equivalente a decir que x << 1.  
Si recordamos el desarrollo en serie de Taylor de la exponencial y nos quedamos en la aproximación a primer orden:

$$ e^x = \sum_{n=1}^{\infty} \frac{x^{n}}{n!} = 1 + x + \frac{x^2}{2} + \cdots \approx 1 + x$$

##### <font color='red'>Para hacer (d)</font>
Llegar a la fórmula de Rayleigh-Jeans mediante la aproximación correspondiente y entregar. Después escribir la función de abajo con la fórmula correpondiente. 

In [ ]:
def rayleigh_jeans_law(nu,T):
    """ Radiance by a Black body at temperature T, as a function of frequency. Low frequency approximation (Rayleigh-Jeans)
    """
    return 0

## Aproximación a alta energía (Ley de Wien)

Se cumplirá para los fotones más energéticos emitidos por el cuerpo negro, para los cuales $h\nu >> kT$. Nuevamente, si hacemos $x = \frac{h\nu}{kT}$ esto es equivalente a decir que x >> 1. 

$$B_{W}(\nu, T) = \frac{2h\nu^3}{c^2} e^{-h\nu/kT} $$

##### <font color='red'>Para hacer (d)</font>
Llegar a la fórmula de Wien mediante la aproximación correspondiente y entregar. Después escribir la función abajo.

In [ ]:
def wien_law(nu,T):
    """ Radiance by a Black body at temperature T, as a function of frequency. High frequency approximation (Wien)
    """
    return 0

##### <font color='red'>Para hacer (d)</font>

Probar con varios valores de longitud de onda para ver cómo se parecen las aproximaciones a la función de Planck

In [ ]:
# Probemos que las funciones funcionan y tienen las unidades correctas

lambda_test = 850*u.nm
nu_test = c/lambda_test

B_P = planck_law(nu_test,T_mystar).to('W / (m2 Hz sr)')
B_RJ =  rayleigh_jeans_law(nu_test,T_mystar).to('W / (m2 Hz sr)')
B_W = wien_law(nu_test,T_mystar).to('W / (m2 Hz sr)')

print(f"Wien          : {B_W}")
print(f"Planck        : {B_P}")
print(f"Rayleigh-Jeans: {B_RJ}")

### Comparación de las aproximaciones

Vamos a determinar, para la emisión de nuestro cuerpo negro ideal, en qué rangos podemos utilizar cada una de las aproximaciones que hemos visto.

Primero representamos gráficamente las tres funciones en función de la frecuencia. 

##### <font color='red'>Para hacer (e)</font>

Probar con varios valores de longitud de onda para ver cómo se parecen las aproximaciones a la función de Planck. Se pueden ver los números que dan las funciones en la celda anterior (evaluados en diferentes frecuencias) para poder tener una idea de los límites del gráfico.

In [ ]:
# Generamos vectores de frecuencia que nos sirvan para una escala logrítmica
nu = generate_nu_vector()

In [ ]:
nu

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

plt.plot(nu.to('THz'),planck_law(nu,T_mystar), color='orange', label=f'T={T_mystar} (Planck)',)
plt.plot(nu.to('THz'),rayleigh_jeans_law(nu,T_mystar), color='violet', label=f'T={T_mystar} (R-J)',ls='--',)
plt.plot(nu.to('THz'),wien_law(nu,T_mystar),color='red', label=f'T={T_mystar} (Wien)',ls=':',)

ax.set_xlabel(r'$\nu$ [THz]',fontsize=14)
ax.set_ylabel(r'Radiance [W m$^{-2}$ Hz$^{-1}$ sr$^{-1}$ ]',fontsize=14)

plt.legend()
plt.grid(1)

### Descomentar abajo y elegir valores adecuados para poder ver bien la zona de interés.

#freq_lim0 = (0*u.THz).value
#freq_lim1 = (1*u.THz).value
#radiance_lim0 = 0
#radiance_lim1 = 1

#ax.set_xlim([freq_lim0, freq_lim1])
#ax.set_ylim([radiance_lim0, radiance_lim1])


Después vamos a usar dos funciones que nos digan las diferencias entre cada una de las aproximaciones con el valor exacto de Planck. 

In [ ]:
def diff_Planck_RJ(nu,T):
    """ Relative difference (percentage) between R_J approximation and Planck function at a certain frequency
    """
    B_P = planck_law(nu,T)
    B_RJ = rayleigh_jeans_law(nu,T)
    return 100*abs((B_P-B_RJ)/B_P)

def diff_Planck_Wien(nu, T):
    """ Relative difference (percentage) between Wien approximation and Planck function at a certain frequency
    """
    B_P = planck_law(nu,T)
    B_W = wien_law(nu,T)
    return 100*abs((B_P-B_W)/B_P)

##### <font color='red'>Para hacer (f)</font>
En la zona central de frecuencias ninguna de las dos aproximaciones es válida. Tendremos que encontrar los valores límites en las dos direcciones (alta y baja frecuencia) para los cuales estas diferencias están por debajo del 1%.

In [ ]:
# Límite de baja frecuencia
freq_limit_RJ = XXXX*u.THz
limit1 = diff_Planck_RJ(freq_limit1,T_mystar)
limit1

In [ ]:
# Límite de alta frecuencia
freq_limit_Wien = XXXX*u.THz
limit2 = diff_Planck_Wien(freq_limit2,T_mystar)
limit2

Finalmente vamos a graficar las radiancias espectrales en escala logarítmica. Incluiremos líneas verticales marcando los limites calculados anteriormente.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

plt.plot(nu,planck_law(nu,T_mystar),label=f'{T_mystar} (Planck)',color='orange')
plt.plot(nu,rayleigh_jeans_law(nu,T_mystar),label=f'{T_mystar} (R-J)',ls='--',color='violet')
plt.plot(nu,wien_law(nu,T_mystar),label=f'{T_mystar} (Wien)',ls=':',color='red')

# Límites calculados en los que las aproximaciones empiezan a ser cercanas al valor exacto (< 1%)
plt.axvline(freq_limit_RJ.to('Hz').value,ls='--')
plt.axvline(freq_limit_Wien.to('Hz').value,ls='-.')

ax.set_xlabel(r'$\nu$ [Hz]',fontsize=14)
ax.set_ylabel(r'Radiance [W m$^{-2}$ Hz$^{-1}$ sr$^{-1}$]',fontsize=14)

# Definimos un eje secundario horizontal (arriba) donde aparezca también la longitud de onda
sec_ax = ax.secondary_xaxis('top', functions=(nu2lambda, lambda2nu))
sec_ax.set_xlabel(r'$\lambda$ [mm]',fontsize=14)

plt.legend()
plt.yscale('log')
plt.xscale('log')
plt.grid(1)

plt.xlim([(1*u.THz).to('Hz').value, (3500*u.THz).to('Hz').value])
plt.ylim([1e-15,0.6e-5])

## Energía promedio de los fotones emitidos por un cuerpo negro

Vamos a calcular la energía promedio de los fotones del cuerpo negro

$$\langle E_{phot}\rangle = \frac{Energia\;total\;emitida}{Numero\;de\;fotones} = \frac{\int_{0}^{\infty} B_{\nu}(T)\,d\nu}{\int_{0}^{\infty} \frac{B_{\nu}(T)}{h\nu}\,d\nu} $$ 

$$ Energia\;total\;emitida = \int_{0}^{\infty} B_{\nu}(T)\,d\nu = \int_{0}^{\infty} \frac{2h\nu^3}{c^2} \frac{1}{e^{h\nu/kT}-1}\,d\nu $$

$$ Numero\;de\;fotones = \int_{0}^{\infty} \frac{B_{\nu}(T)}{h\nu}\,d\nu = \int_{0}^{\infty} \frac{2\nu^2}{c^2} \frac{1}{e^{h\nu/kT}-1}\,d\nu $$ 

En cambos casos hacemos un cambio de variable $x = \frac{h\nu}{kT}$, tenemos $dx = \frac{h}{kT}d\nu$ y sustituimos arriba.


Para resolver esta integral, podemos hacerlo numéricamente, aunque también podemos echar mano de la función $\zeta$ de Riemann (ver https://en.wikipedia.org/wiki/Riemann_zeta_function):

$$\int_{0}^{\infty} \frac{x^n}{e^x-1} = n!\,\zeta(n+1)$$

Al final llegaremos a la siguiente expresión:

$$ Energia\;total\;emitida = \frac{2h}{c^{2}} \left( \frac{kT}{h}\right)^{4} 3!\,\zeta(4) $$

$$ Numero\;de\;fotones = \frac{2}{c^{2}} \left( \frac{kT}{h}\right)^{3} 2!\,\zeta(3) $$ 

Por tanto:

$$\langle E_{phot}\rangle = kT \frac{3\zeta(4)}{\zeta(3)} $$

##### <font color='red'>Para hacer (g)</font>
Obtener estas expresiones finales para la energía total emitida y para el número de fotones haciendo los cambios de variable indicados anteriormente y dejando las integrales en función de la $\zeta(n)$ con el n que corresponde.

Veamos algo interesante respecto al cuerpo negro: 

   * La energía total (bolométrica) emitida total por el cuerpo negro es proporcional a $T^4$. Esto lo sabemos de la ley de Stefan-Botzmann (ojo, la ley de S-B se refiere al flujo luminoso recibido, no a la radiancia B$_\nu$(T), pero la dependencia con la temperatura es la misma).
   * El número de fotones que se emiten por segundo y por unidad de área es proporcional a $T^3$. 
   * La energía promedio de cada fotón es proporcional a T únicamente.
   
Esto significa que cuando el cuerpo negro aumenta su temperatura ocurre que: (i) se emiten más fotones por segundo; (ii) cada fotón en promedio tendrá mayor energía. Sin embargo el primero de los dos factores domina mucho más que el otro

Ahora calculemos la energía promedio de los fotones $\langle E_{phot}\rangle$ que será un factor constante multiplicando a kT 

In [ ]:
# Calculamos el factor usando la función  Riemann zeta function:

factor = 3*zeta(4)/zeta(3)
print(factor)

In [ ]:
# Se puede llegar al mismo factor haciendo las integrales correspondientes de forma numérica con la función *quad* de scipy.
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html
# Esta función utiliza una librería en Fortran llamada QUADPACK. Para detalles ver https://en.wikipedia.org/wiki/QUADPACK
#
# Debemos definir funciones con los integrandos y poner los límites de integración (en este caso entre 0 e infinito).

factor2 = quad(integrand3, 0, np.Inf)[0] / quad(integrand2, 0, np.Inf)[0]
print(factor2)

##### <font color='red'>Para hacer (h)</font>

Calcular la energía promedio según la formula que corresponde y expresar en eV.

In [ ]:
E_mean = factor * XXX
# Convertir a las unidades que se piden (eV)


Finalmente calculamos el número de fotones por segundo, por m$^2$ y por esteroradián

In [ ]:
N_phot = np.pi*2/np.power(c,2)*pow((k_B)*T_mystar/h,3)*2*zeta(3)/u.sr
N_phot.to(' / (s m2 sr)')

La energía total radiada (por segundo, por unidad de área) se puede calcular simplemente como el producto del número de fotones por la energía promedio de los fotones:

$$E_{tot} = N_{phot}\,\langle E_{phot}\rangle $$

In [ ]:
E_tot = N_phot * E_mean
E_tot.to("W / (m2 sr)")